In [3]:
from dotenv import dotenv_values
from requests import get
from requests.models import HTTPError
from bs4 import BeautifulSoup

import pandas as pd

In [4]:
CONFIG = dotenv_values('.env')

In [5]:
def requisicao_api_alura():
    try:
        response = get('https://cursos.alura.com.br/api/categorias')
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Um erro HTTP ocorreu: {http_err}')
    except Exception as err:
        print(f'Um erro ocorreu: {err}')
    else:
        return response.json()

In [6]:
def requisicao(url):
    cookies = {'caelum.login.token':CONFIG['COOKIE']}
    try:
        response = get(url, cookies=cookies)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Um erro HTTP ocorreu: {http_err}')
    except Exception as err:
        print(f'Um erro ocorreu: {err}')
    else:
        soup = BeautifulSoup(response.text,'html.parser')
        return soup

In [15]:
def cursos():
    dados = requisicao_api_alura()
    url = 'https://cursos.alura.com.br/forum/curso-'
    nomes_cursos = []
    nomes_categorias = []
    urls_cursos = []
    for item in dados:
        for curso in item['cursos']:
            nomes_cursos.append(curso['nome'])
            nomes_categorias.append(item['nome'])
            urls_cursos.append(f'{url}{curso["slug"]}/sem-resposta')
    df = pd.DataFrame({'Curso': nomes_cursos,'Categoria':nomes_categorias, 'Url': urls_cursos})
    return df

In [12]:
dados = requisicao_api_alura()

In [16]:
df = cursos()

In [17]:
df

,Curso,Categoria,Url
0,Estrutura de Dados: Computação na prática com ...,Programação,https://cursos.alura.com.br/forum/curso-estrut...
1,Spring Boot API REST: Construa uma API,Programação,https://cursos.alura.com.br/forum/curso-spring...
2,C I: Introdução à Linguagem das Linguagens,Programação,https://cursos.alura.com.br/forum/curso-introd...
3,C# parte 1: Primeiros passos,Programação,https://cursos.alura.com.br/forum/curso-csharp...
4,Design Patterns C# I: Boas práticas de program...,Programação,https://cursos.alura.com.br/forum/curso-design...
...,...,...,...
1267,Gerenciamento de estresse: Otimize sua liderança,Inovação & Gestão,https://cursos.alura.com.br/forum/curso-gerenc...
1268,Comunicação participativa: Liderança eficaz e ...,Inovação & Gestão,https://cursos.alura.com.br/forum/curso-comuni...
1269,Comunicação estratégica na liderança: Como eng...,Inovação & Gestão,https://cursos.alura.com.br/forum/curso-comuni...
1270,Negociação para líderes: Desenvolva a habilida...,Inovação & Gestão,https://cursos.alura.com.br/forum/curso-negoci...


In [18]:
def roda_scraping_cursos(df):
    topicos_sem_resposta = []
    for url in df['Url']:
        soup = requisicao(url)
        paginacao = soup.find('nav', {'class': 'busca-paginacao-links'})
        qtd_topicos = 0
        if paginacao:
            ultima_pagina = paginacao.get_text().split()[-1]
            qtd_topicos += (int(ultima_pagina)-1)*20
            url += f'/{ultima_pagina}'
            soup = requisicao(url)
        qtd_topicos += len(soup.find_all('div', {'class': {'forumList-item-subject'}}))
        topicos_sem_resposta.append(qtd_topicos)
    df['Tópicos sem resposta'] = topicos_sem_resposta
    return df

In [20]:
dados = roda_scraping_cursos(df)
dados

,Curso,Categoria,Url,Tópicos sem resposta
0,Estrutura de Dados: Computação na prática com ...,Programação,https://cursos.alura.com.br/forum/curso-estrut...,7
1,Spring Boot API REST: Construa uma API,Programação,https://cursos.alura.com.br/forum/curso-spring...,2
2,C I: Introdução à Linguagem das Linguagens,Programação,https://cursos.alura.com.br/forum/curso-introd...,10
3,C# parte 1: Primeiros passos,Programação,https://cursos.alura.com.br/forum/curso-csharp...,4
4,Design Patterns C# I: Boas práticas de program...,Programação,https://cursos.alura.com.br/forum/curso-design...,15
...,...,...,...,...
1267,Gerenciamento de estresse: Otimize sua liderança,Inovação & Gestão,https://cursos.alura.com.br/forum/curso-gerenc...,0
1268,Comunicação participativa: Liderança eficaz e ...,Inovação & Gestão,https://cursos.alura.com.br/forum/curso-comuni...,0
1269,Comunicação estratégica na liderança: Como eng...,Inovação & Gestão,https://cursos.alura.com.br/forum/curso-comuni...,0
1270,Negociação para líderes: Desenvolva a habilida...,Inovação & Gestão,https://cursos.alura.com.br/forum/curso-negoci...,0


In [21]:
pd.options.display.max_rows = 999

In [25]:
dados[dados['Categoria']=='Data Science'].sort_values(['Tópicos sem resposta'], ascending = False)

,Curso,Categoria,Url,Tópicos sem resposta
422,Machine Learning: Introdução a classificação c...,Data Science,https://cursos.alura.com.br/forum/curso-machin...,19
574,Corretor Ortográfico em Python: Aplicando técn...,Data Science,https://cursos.alura.com.br/forum/curso-nlp-co...,17
471,Machine Learning: Classificação por trás dos p...,Data Science,https://cursos.alura.com.br/forum/curso-introd...,11
578,Modelos preditivos em dados: detecção de fraude,Data Science,https://cursos.alura.com.br/forum/curso-modelo...,10
480,Data Science: Introdução a análise de series t...,Data Science,https://cursos.alura.com.br/forum/curso-data-s...,10
545,GeoPandas Parte 1: Trabalhando com dados Geoes...,Data Science,https://cursos.alura.com.br/forum/curso-geopan...,10
544,Deep Learning parte 2: Como a rede aprende,Data Science,https://cursos.alura.com.br/forum/curso-deep-l...,8
459,Data Science: Introdução a testes estatísticos...,Data Science,https://cursos.alura.com.br/forum/curso-data-s...,7
559,Análise de dados: Introdução com R,Data Science,https://cursos.alura.com.br/forum/curso-busine...,7
541,Deep Learning parte 1: Introdução com Keras,Data Science,https://cursos.alura.com.br/forum/curso-deep-l...,7
